In [43]:
import json
import os
import IPython.display as ipd
import time

from glob import glob
from os.path import splitext, join as pjoins
from itertools import chain
from shutil import copyfile
from tqdm import tqdm
from copy import deepcopy
# import xenocanto
import pandas as pd
import numpy as np
import librosa
import soundfile as SF
from shutil import copyfile
from joblib import delayed

from code_base.utils.audio_utils import get_audio_metadata
from code_base.utils import load_json, write_json
from code_base.utils.main_utils import download_audio_file
from code_base.utils.main_utils import ProgressParallel

tqdm.pandas()

# Upload data

In [2]:
!ls ../data/inaturalist_online/ -lt

total 4896
-rw-rw-r-- 1 vsydorskyi vsydorskyi 1579953 тра 26 15:30 raw_meta_2025_part2.csv
drwxrwxr-x 6 vsydorskyi vsydorskyi    4096 тра 26 15:17 audio
-rw-rw-r-- 1 vsydorskyi vsydorskyi 1560285 тра 26 00:39 raw_meta_part3.csv
-rw-rw-r-- 1 vsydorskyi vsydorskyi  121704 тра 25 17:05 final_sorted_inat_meta.csv
-rw-r--r-- 1 vsydorskyi vsydorskyi  153373 тра 25 16:44 raw_meta_2025_part1.csv
-rw-rw-r-- 1 vsydorskyi vsydorskyi 1580792 тра 19 17:13 raw_meta_part2.csv
-rw-rw-r-- 1 vsydorskyi vsydorskyi    2395 тра 14 20:33 download_species_2025_part2.json
-rw-rw-r-- 1 vsydorskyi vsydorskyi    2462 тра 14 20:33 download_species_2025_part3.json


In [3]:
inat_meta_artur = pd.read_csv(
    "../data/inaturalist_online/final_sorted_inat_meta.csv"
)
inat_meta_vova_p1 = pd.read_csv(
    "../data/inaturalist_online/raw_meta_2025_part1.csv"   
)
inat_meta_vova_p2 = pd.read_csv(
    "../data/inaturalist_online/raw_meta_part2.csv"   
)
inat_meta_vova_hpc_p2 = pd.read_csv(
    "../data/inaturalist_online/raw_meta_2025_part2.csv"   
)
inat_meta_vova_p3 = pd.read_csv(
    "../data/inaturalist_online/raw_meta_part3.csv"   
)

In [4]:
train_metadata = pd.read_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa_fixedaudiometa_h5pyDur.csv"
)

birds_2025 = set(load_json(
    "../data/bird2int_2025.json"
).keys())

/tmp/ipykernel_395868/1446142774.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv(


In [5]:
eBird_Taxonomy_v2021 = pd.read_csv("../data/taxonomy.csv")

eBird_Taxonomy_v2021["scientific_name"] = eBird_Taxonomy_v2021["scientific_name"].str.lower()
eBird_Taxonomy_v2021["common_name"] = eBird_Taxonomy_v2021["common_name"].str.lower()

# Enrich taxonomy with previous year taxonomy
# It was checked that there are no conflicts

taxonomy_2024 = pd.read_csv(
    "../data/eBird_Taxonomy_v2021.csv"
)
taxonomy_2024 = taxonomy_2024.rename(columns={
    "SPECIES_CODE": "primary_label",
    "PRIMARY_COM_NAME": "common_name",
    "SCI_NAME": "scientific_name"
})[["primary_label", "common_name", "scientific_name"]]

taxonomy_2024["scientific_name"] = taxonomy_2024["scientific_name"].str.lower()
taxonomy_2024["common_name"] = taxonomy_2024["common_name"].str.lower()

eBird_Taxonomy_v2021 = pd.concat([
    eBird_Taxonomy_v2021,
    taxonomy_2024
])

eBird_Taxonomy_v2021 = eBird_Taxonomy_v2021.drop_duplicates("primary_label")

# Check duplicates

In [6]:
train_metadata["collection"].value_counts()

collection
XC              28840
iNat             7198
CSA_11052025     3004
CSA               162
Name: count, dtype: int64

In [7]:
# inat_meta = pd.concat([inat_meta_vova_p1, inat_meta_vova_p2, inat_meta_vova_hpc_p2, inat_meta_vova_p3]).reset_index(drop=True)
inat_meta = pd.concat([inat_meta_vova_p1, inat_meta_vova_p2, inat_meta_vova_p3]).reset_index(drop=True)

In [8]:
inat_meta["license"].value_counts(dropna=False)

license
cc-by-nc       9091
NaN            4515
cc-by          1426
cc-by-nc-nd     260
cc-by-nc-sa     206
cc0             205
cc-by-sa        184
cc-by-nd          1
Name: count, dtype: int64

In [9]:
inat_meta_vova_hpc_p2["license"].value_counts(dropna=False)

license
cc-by-nc       4535
NaN            2221
cc-by           596
cc0             118
cc-by-nc-nd     112
cc-by-nc-sa      93
cc-by-sa         79
cc-by-nd          1
Name: count, dtype: int64

In [10]:
inat_meta_artur["license"].value_counts(dropna=False)

license
NaN            361
cc-by-nc       140
cc-by-nc-nd     22
cc-by           20
cc-by-nc-sa      3
Name: count, dtype: int64

In [11]:
inat_meta = inat_meta[~inat_meta["license"].isna()].reset_index(drop=True)
inat_meta = inat_meta[~inat_meta["license"].isin(["cc-by-nd"])].reset_index(drop=True)

In [12]:
inat_meta_artur = inat_meta_artur[~inat_meta_artur["license"].isna()].reset_index(drop=True)

In [13]:
inat_meta_vova_hpc_p2 = inat_meta_vova_hpc_p2[~inat_meta_vova_hpc_p2["license"].isna()].reset_index(drop=True)
inat_meta_vova_hpc_p2 = inat_meta_vova_hpc_p2[~inat_meta_vova_hpc_p2["license"].isin(["cc-by-nd"])].reset_index(drop=True)

In [14]:
inat_meta["normalize_url"] = inat_meta["url"].apply(lambda x: os.path.splitext(x.split("?")[0])[0])
inat_meta_vova_hpc_p2["normalize_url"] = inat_meta_vova_hpc_p2["url"].apply(lambda x: os.path.splitext(x.split("?")[0])[0])
inat_meta_artur["normalize_url"] = inat_meta_artur["url"].apply(lambda x: os.path.splitext(x.split("?")[0])[0])

In [15]:
def drop_duplicate_id_files(input_df):
    url_vc = input_df["normalize_url"].value_counts()
    drop_files = url_vc[url_vc > 1].index.to_list()
    print(f"Dropping {url_vc[url_vc > 1].sum()} rows")
    return input_df[~input_df["normalize_url"].isin(drop_files)].reset_index(drop=True)

In [16]:
inat_meta = drop_duplicate_id_files(inat_meta)
inat_meta_vova_hpc_p2 = drop_duplicate_id_files(inat_meta_vova_hpc_p2)
inat_meta_artur = drop_duplicate_id_files(inat_meta_artur)

Dropping 73 rows
Dropping 15 rows
Dropping 0 rows


In [17]:
inat_meta["normalize_url"].value_counts()

normalize_url
https://static.inaturalist.org/sounds/141983     1
https://static.inaturalist.org/sounds/1144905    1
https://static.inaturalist.org/sounds/1348913    1
https://static.inaturalist.org/sounds/1345030    1
https://static.inaturalist.org/sounds/1341703    1
                                                ..
https://static.inaturalist.org/sounds/277243     1
https://static.inaturalist.org/sounds/270665     1
https://static.inaturalist.org/sounds/264730     1
https://static.inaturalist.org/sounds/258356     1
https://static.inaturalist.org/sounds/33510      1
Name: count, Length: 11299, dtype: int64

In [18]:
inat_meta = pd.concat([inat_meta, inat_meta_vova_hpc_p2]).reset_index(drop=True)
inat_meta = inat_meta.drop_duplicates("normalize_url").reset_index(drop=True)

In [19]:
inat_meta["normalize_url"].value_counts()

normalize_url
https://static.inaturalist.org/sounds/141983     1
https://static.inaturalist.org/sounds/1375828    1
https://static.inaturalist.org/sounds/1280999    1
https://static.inaturalist.org/sounds/1279022    1
https://static.inaturalist.org/sounds/1276244    1
                                                ..
https://static.inaturalist.org/sounds/193974     1
https://static.inaturalist.org/sounds/181565     1
https://static.inaturalist.org/sounds/173522     1
https://static.inaturalist.org/sounds/172894     1
https://static.inaturalist.org/sounds/1275982    1
Name: count, Length: 11329, dtype: int64

In [20]:
inat_species_from_2025 = set(train_metadata["scientific_name"].str.lower()) & set(inat_meta["primary_label"].str.lower())
inat_species_from_general_taxonomy = set(inat_meta["primary_label"].str.lower()) & set(eBird_Taxonomy_v2021["scientific_name"])

print(f"N species from 2025 {len(inat_species_from_2025)} out of {len(set(inat_meta['primary_label']))}")
print(f"N species from Gen Taxonomy {len(inat_species_from_general_taxonomy)} out of {len(set(inat_meta['primary_label']))}")

N species from 2025 187 out of 187
N species from Gen Taxonomy 187 out of 187


In [21]:
sn2pl = eBird_Taxonomy_v2021.set_index("scientific_name")["primary_label"].to_dict()
inat_meta["primary_label"] = inat_meta["primary_label"].str.lower().map(sn2pl)

In [22]:
inat_meta = pd.concat([inat_meta, inat_meta_artur[inat_meta.columns]]).reset_index(drop=True)
inat_meta = inat_meta.drop_duplicates("normalize_url").reset_index(drop=True)

In [23]:
inat_meta["normalize_url"].value_counts()

normalize_url
https://static.inaturalist.org/sounds/141983     1
https://static.inaturalist.org/sounds/1341703    1
https://static.inaturalist.org/sounds/1271625    1
https://static.inaturalist.org/sounds/1271302    1
https://static.inaturalist.org/sounds/1269905    1
                                                ..
https://static.inaturalist.org/sounds/172894     1
https://static.inaturalist.org/sounds/171840     1
https://static.inaturalist.org/sounds/171839     1
https://static.inaturalist.org/sounds/169314     1
https://static.inaturalist.org/sounds/1160095    1
Name: count, Length: 11338, dtype: int64

In [24]:
duplicate_values = set(train_metadata.loc[train_metadata["collection"] == "iNat", "url"].apply(lambda x: os.path.splitext(x)[0]))

inat_meta = inat_meta[~inat_meta["normalize_url"].isin(duplicate_values)].reset_index(drop=True)
inat_meta = inat_meta.drop(columns=["normalize_url"])

In [25]:
inat_meta.shape

(6492, 13)

In [26]:
inat_meta = inat_meta.reset_index().rename(columns={"index": "id"})
inat_meta["id"] += 100000_000

In [27]:
inat_meta["filename"] = inat_meta.apply(lambda row: os.path.join(row["primary_label"], str(row["id"]) + ".wav"), axis=1)

In [28]:
assert not set(inat_meta["filename"].apply(lambda x: os.path.splitext(x)[0])) & set(train_metadata["filename"].apply(lambda x: os.path.splitext(x)[0]))

# Download Files

In [29]:
inat_meta["full_filename"] = inat_meta["filename"].apply(lambda x: os.path.join(
    "../data/inaturalist_online/audio", x
))

In [48]:
import requests

def download_audio_file(url: str, save_path: str, verbose: bool = True):
    """
    Downloads an audio file from the provided URL and saves it to the specified path.

    Args:
        url (str): Direct download link to the audio file.
        save_path (str): Full path including filename where the audio should be saved.
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise error for bad status codes
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        if verbose:
            print(f"Audio file downloaded successfully to {save_path}")
        return True
    except Exception as e:
        if verbose:
            print(f"Error downloading audio file: {e}")
        return False

def download_and_create_folder(
    url,
    save_path_
):
    if os.path.exists(save_path_):
        return
    if not os.path.exists(os.path.dirname(save_path_)):
        os.makedirs(os.path.dirname(save_path_), exist_ok=True)
    download_audio_file(url, save_path_, verbose=False)    

In [31]:
ProgressParallel(
    n_jobs=4,
    total=len(inat_meta),
    backend="threading",
)(delayed(download_and_create_folder)(download_link, save_path) for download_link, save_path in zip(inat_meta["url"], inat_meta["full_filename"]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6492/6492 [24:25<00:00,  4.43it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [32]:
# Enrich with audio meta

audio_meta = pd.DataFrame(inat_meta["full_filename"].progress_apply(get_audio_metadata).to_list())

inat_meta = pd.concat([
    inat_meta, audio_meta
], axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6492/6492 [00:50<00:00, 129.01it/s]


In [40]:
inat_meta_to_load = inat_meta[inat_meta["sample_rate"].isna()].reset_index(drop=True)

In [50]:
for download_link, save_path in zip(inat_meta_to_load["url"], inat_meta_to_load["full_filename"]):
    is_downloaded = download_audio_file(download_link, save_path)
    if not is_downloaded:
        print("Download Failed. Sleeping for 30 seconds ....")
        time.sleep(30)
        is_downloaded = download_audio_file(download_link, save_path)

Audio file downloaded successfully to ../data/inaturalist_online/audio/65448/100001872.wav
True
Audio file downloaded successfully to ../data/inaturalist_online/audio/65962/100001892.wav
True
Audio file downloaded successfully to ../data/inaturalist_online/audio/66893/100001914.wav
True
Audio file downloaded successfully to ../data/inaturalist_online/audio/blhpar1/100002565.wav
True
Audio file downloaded successfully to ../data/inaturalist_online/audio/cocwoo1/100002992.wav
True
Audio file downloaded successfully to ../data/inaturalist_online/audio/grbhaw1/100003440.wav
True
Audio file downloaded successfully to ../data/inaturalist_online/audio/orcpar/100004179.wav
True
Error downloading audio file: 404 Client Error: Not Found for url: https://www.inaturalist.org/files/original/missing.png
False
Download Failed. Sleeping for 30 seconds ....
Error downloading audio file: 404 Client Error: Not Found for url: https://www.inaturalist.org/files/original/missing.png
Audio file downloaded suc

In [52]:
audio_meta = pd.DataFrame(inat_meta["full_filename"].progress_apply(get_audio_metadata).to_list())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6492/6492 [00:58<00:00, 111.21it/s]


In [59]:
inat_meta = inat_meta.drop(columns=audio_meta.columns)

In [60]:
inat_meta = pd.concat([
    inat_meta, audio_meta
], axis=1)

In [62]:
inat_meta["sample_rate"].value_counts(dropna=False)

sample_rate
44100.0     4323
48000.0     1863
8000.0        74
192000.0      57
24000.0       48
32000.0       40
22050.0       31
96000.0       22
16000.0       19
NaN           10
11025.0        2
88200.0        2
250000.0       1
Name: count, dtype: int64

In [34]:
# inat_meta.to_csv(
#     "../data/inaturalist_online/merged_meta_drop_dupls.csv",
#     index=False
# )

In [63]:
inat_meta = inat_meta[~inat_meta["sample_rate"].isna()].reset_index(drop=True)

# Correct Columns

In [66]:
print(f"New columns: {set(inat_meta.columns) - set(train_metadata.columns)}")
print(f"Lack columns: {set(train_metadata.columns) - set(inat_meta.columns)}")

New columns: {'full_filename'}
Lack columns: {'dataset', 'data_root_id', 'duration_s_h5py'}


In [69]:
inat_meta = inat_meta.drop(columns=["full_filename"])

inat_meta["collection"] = "iNat_26052025"
inat_meta["data_root_id"] = "inat_train_audio_parsed_26052025"
inat_meta["secondary_labels"] = [[] for _ in range(len(inat_meta))]
inat_meta["dataset"] = "csa_11052025"

In [82]:
import h5py

def get_h5dur(fname):
    with h5py.File(fname, "r", swmr=True) as f:
        real_dur = len(f["au"]) / 32000
    return real_dur
inat_meta["duration_s_h5py"] = inat_meta.apply(
    lambda row: os.path.join("../data/inat_train_features_parsed_26052025", row["primary_label"], str(row["id"]) + ".hdf5"), 
    axis=1
).apply(get_h5dur)

In [83]:
print(f"New columns: {set(inat_meta.columns) - set(train_metadata.columns)}")
print(f"Lack columns: {set(train_metadata.columns) - set(inat_meta.columns)}")

New columns: set()
Lack columns: set()


In [84]:
inat_meta = inat_meta[train_metadata.columns]

In [88]:
inat_meta.to_csv(
    "../data/inaturalist_online/merged_meta_drop_dupls_prepared.csv",
    index=False
)

In [89]:
train_metadata_with_inat = pd.concat([train_metadata, inat_meta]).reset_index(drop=True)

In [90]:
train_metadata_with_inat.to_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa_inat_fixedaudiometa_h5pyDur.csv",
    index=False
)

In [91]:
train_metadata = pd.read_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5_fixedaudiometa_h5pyDur.csv"
)

In [92]:
train_metadata_with_inat = pd.concat([train_metadata, inat_meta]).reset_index(drop=True)

In [93]:
train_metadata_with_inat["data_root_id"].value_counts()

data_root_id
train_audio                                 28564
add_train_audio_from_xeno_canto_28032025     7376
inat_train_audio_parsed_26052025             6482
add_train_audio_from_prev_comps                90
Name: count, dtype: int64

In [94]:
train_metadata_with_inat.to_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5_inat_fixedaudiometa_h5pyDur.csv",
    index=False
)

# Split into Train and Pretrain

In [ ]:
csa_meta_selected_train = csa_meta_selected[csa_meta_selected["primary_label"].isin(birds_2025)].reset_index(drop=True)

In [ ]:
csa_meta_selected_train["filename"]

In [ ]:
for fname in tqdm(csa_meta_selected_train["filename"]):
    source_fname = os.path.join(
        "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/csa/audio", fname
    )
    target_fname = os.path.join(
        "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/csa/csa_train_audio_parsed_11052025", fname
    )
    if not os.path.exists(os.path.dirname(target_fname)):
        os.makedirs(os.path.dirname(target_fname))
    copyfile(
        source_fname,
        target_fname
    )

In [ ]:
csa_meta_selected_train.to_csv(
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/csa/CSA_meta_extended_nodupl_train.csv",
    index=False
)

In [ ]:
train_metadata_with_csa = pd.concat([train_metadata, csa_meta_selected_train], axis=0).reset_index(drop=True)

In [ ]:
train_metadata_with_csa.to_csv(
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa.csv",
    index=False
)

In [ ]:
train_metadata_with_csa["data_root_id"].value_counts()

# Split into Train and Pretrain

In [ ]:
csa_meta_selected_pretrain = csa_meta_selected[~csa_meta_selected["primary_label"].isin(birds_2025)].reset_index(drop=True)

In [ ]:
csa_meta_selected_pretrain.shape

In [ ]:
for fname in tqdm(csa_meta_selected_pretrain["filename"]):
    source_fname = os.path.join(
        "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/csa/audio", fname
    )
    target_fname = os.path.join(
        "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/csa/csa_pretrain_audio_parsed_11052025", fname
    )
    if not os.path.exists(os.path.dirname(target_fname)):
        os.makedirs(os.path.dirname(target_fname))
    copyfile(
        source_fname,
        target_fname
    )

In [ ]:
csa_meta_selected_pretrain.to_csv(
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/csa/CSA_meta_extended_nodupl_pretrain.csv",
    index=False
)

In [ ]:
general_pretrain = pd.read_csv(
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/xeno_canto/data_all_years_and_more/dataset/train_and_prev_comps_extendedv1_pruneSL_XCallyearstaxonomy_snipet11052025_hdf5.csv",
)

In [ ]:
general_pretrain_with_csa = pd.concat([general_pretrain, csa_meta_selected_pretrain], axis=0).reset_index(drop=True)

In [ ]:
general_pretrain_with_csa.to_csv(
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/xeno_canto/data_all_years_and_more/dataset/train_and_prev_comps_extendedv1_pruneSL_XCallyearstaxonomy_snipet11052025_csa.csv",
    index=False
)

In [ ]:
general_pretrain_with_csa["data_root_id"].value_counts()